# Utilities for restacking pipeline

This notebook is a collection of various utility functions that can be used to assist with running the pipeline. 

Run the cell below to set up the environment before moving on to other parts of the notebook.

In [3]:
from config import *
import luts
import restack_20km as main

years = luts.groups[group]["years"]
wrf_dir = Path(luts.groups[group]["directory"])

### Inspect the filesystem

This section just provides some info on the system we are working with here.

The WRF outputs of interest from different runs of model / scenario may be in separate directories, but there is consistency in file structure across all groups - all `hourly` and `daily` directories have annual subgroups consisting of the WRF outputs to be restacked:

In [2]:
ls /archive/DYNDOWN/DIONE/pbieniek/ccsm/hist/hourly | head -5

1970/
1971/
1972/
1973/
1974/


In [14]:
ls /archive/DYNDOWN/DIONE/pbieniek/ccsm/hist/hourly/ | tail -6

2003/
2004/
2005/
nohup.out
orgdata.sh*


In [18]:
ls /archive/DYNDOWN/DIONE/pbieniek/ccsm/hist/hourly/1979 | head -5

dailylog.out*
WRFDS_d01.1979-01-01_00.nc*
WRFDS_d01.1979-01-01_01.nc*
WRFDS_d01.1979-01-01_02.nc*
WRFDS_d01.1979-01-01_03.nc*
ls: write error


This structure applies for all outputs, and exists for the following model / scenario / year combinations:

* ERA-Interim
    * "historical": 1979-2015
* GFDL-CM3
    * historical: 1970-2006
    * RCP 8.5: 2006-2100
* NCAR-CCSM4
    * historical: 1970-2005
    * RCP 8.5: 2005-2100

#### System

This pipeline is being developed on the Chinook cluster:

In [1]:
!uname -a

Linux chinook00.rcs.alaska.edu 2.6.32-754.35.1.el6.61015g0000.x86_64 #1 SMP Mon Dec 21 12:41:07 EST 2020 x86_64 x86_64 x86_64 GNU/Linux


This pipeline makes use of slurm and multiple cores / compute nodes for processing in reasonable time.

In [5]:
!sinfo -V

slurm 19.05.7


### Ensure ancillary WRF geogrid file is present

The restacking will rely on a WRF geogrid data file for determining correct spatial projection information, and for correctly rotating data for wind variables. Make sure that it is present in the `anc_dir` directory:

In [ ]:
if not geogrid_fp.exists():
    # the original location of this file is not known, in case it is ever deleted
    #  from this source location it might still be available on Poseidon at 
    #  /workspace/Shared/Tech_Projects/wrf_data/project_data/ancillary_wrf_constants/geo_em.d01.nc
    shutil.copy("/import/SNAP/wrf_data/project_data/ancillary_wrf_constants/geo_em.d01.nc", geogrid_fp)

### Check progress of copying to scratch

Check to see how many of the expected hourly WRF outputs have successfully been copied to scratch space from `$ARCHIVE`. This can take a while.

**Note** - only do this if files you are trying to copy have actually been staged. It should theoretically still work regardless, but will take an unknown amount of time.

In [5]:
wrf_fps, existing_scratch_fps = main.check_raw_scratch(wrf_dir, group, years, raw_scratch_dir)

### Tweak the spatial ref of some processed files

Okay I've just discovered that somehow the spatial reference specification in the restacking pipeline has a bug in it. We have processed the CCSM4 Historical and some of the RCP 8.5 data at this point. Literally just need to change the `grid_mapping` attribute value of the main data variable for each file - (it needs to be `spatial_ref`, the name of the CRS coordinate variable, instead of `polar_stereographic`. Have verified doing this on some test files using the `ncatted` command (built-in). Going with this route to avoid having to re-stage/copy/process all of the processed data. Using xarray to modify parts of netCDF files in place is apparently not straightforward. 

In [2]:
from config import *
import luts
import xarray as xr

Iterate over the variable names and make sure the grid mapping attribute of the main data variable of the file is "spatial_ref". Start with hourly:

In [17]:
command_prefix = "ncatted -O -a grid_mapping,{},o,c,spatial_ref "

commands = []
for varname in luts.varnames:
    var_fps = output_dir.joinpath("hourly", varname.lower()).glob("*.nc")
    commands += [command_prefix.format(varname.lower()) + str(fp) for fp in var_fps]

In [30]:
%%time
# run the commands:
_ = [os.system(command) for command in commands]

CPU times: user 159 ms, sys: 11.3 s, total: 11.5 s
Wall time: 19min 43s


Then daily:

In [31]:
commands = []
for varname in luts.resample_varnames:
    var_fps = output_dir.joinpath("daily", varname.lower()).glob("*.nc")
    commands += [command_prefix.format(varname.lower()) + str(fp) for fp in var_fps]

In [35]:
%%time
_ = [os.system(command) for command in commands]

CPU times: user 9 ms, sys: 1.24 s, total: 1.25 s
Wall time: 48.2 s
